In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('lyrics.csv')

df['genre'] = np.where(
    (df['genre'] == 'Pop')|
    (df['genre'] == 'Country')|
    (df['genre'] == 'Hip-Hop'), 
    'Rock', 
    df['genre']
)


In [2]:
group = ['song', 'year','genre', 'artist', 'lyrics']
lyrics_by_song = df.sort_values(group)\
        .groupby(group).lyrics\
        .apply(' '.join)\
        .apply(lambda x: x.lower())\
        .reset_index(name='lyric')
lyrics_by_song["lyric"] = lyrics_by_song['lyric'].str.replace(r'[^\w\s]','')

In [3]:
lyrics_by_song.head()

,song,year,genre,artist,lyrics,lyric
0,0-0,2006,Metal,fear-factory,Long live the new flesh!\nWhere evil dwells\nN...,long live the new flesh\nwhere evil dwells\nno...
1,0-0-0,2007,Metal,earthtone9,thousand rays\nthousand leaves\nthousand breat...,thousand rays\nthousand leaves\nthousand breat...
2,0-0-where-evil-dwells,2008,Metal,fear-factory,Long live the new flesh\nWhere evil dwells\nNo...,long live the new flesh\nwhere evil dwells\nno...
3,0-71,2006,Not Available,cash-roseanne,no lyrics just 71 seconds of silence for each ...,no lyrics just 71 seconds of silence for each ...
4,0-to-100,2014,Rock,drake,Fuck bein' on some chill shit\nWe go 0 to 100 ...,fuck bein on some chill shit\nwe go 0 to 100 n...


In [4]:
lyrics_by_song2 = lyrics_by_song[['song', 'year', 'artist', 'lyric', 'genre']]

In [5]:
lyrics_by_song2.head()

,song,year,artist,lyric,genre
0,0-0,2006,fear-factory,long live the new flesh\nwhere evil dwells\nno...,Metal
1,0-0-0,2007,earthtone9,thousand rays\nthousand leaves\nthousand breat...,Metal
2,0-0-where-evil-dwells,2008,fear-factory,long live the new flesh\nwhere evil dwells\nno...,Metal
3,0-71,2006,cash-roseanne,no lyrics just 71 seconds of silence for each ...,Not Available
4,0-to-100,2014,drake,fuck bein on some chill shit\nwe go 0 to 100 n...,Rock


In [6]:
lyrics_by_song2['tokens'] = lyrics_by_song2.lyric.apply(lambda x: x[0:-1].replace('\n',' ').split(' '))

In [7]:
lyrics_by_song3 = lyrics_by_song2.loc[(lyrics_by_song2['genre'] == 'Hip Hop') | (lyrics_by_song2['genre'] == 'Pop') |
                                      (lyrics_by_song2['genre'] == 'Rock') | (lyrics_by_song2['genre'] == 'Country')]
lyrics_by_song3.head()
                                      

,song,year,artist,lyric,genre,tokens
4,0-to-100,2014,drake,fuck bein on some chill shit\nwe go 0 to 100 n...,Rock,"[fuck, bein, on, some, chill, shit, we, go, 0,..."
5,0-to-100-freestyle,2014,ace-hood,better watch your mrs imma shoplift\ngo to sla...,Rock,"[better, watch, your, mrs, imma, shoplift, go,..."
6,0-to-100-freestyle,2014,cassidy,imma keep it a bean pinto\nlife is a bitch but...,Rock,"[imma, keep, it, a, bean, pinto, life, is, a, ..."
7,0-to-100-the-catch-up,2014,drake,fuck bein on some chill shit\nwe go 0 to 100 n...,Rock,"[fuck, bein, on, some, chill, shit, we, go, 0,..."
8,000-000,2006,beck,all the way\npixel in the light\ncomin next ye...,Rock,"[all, the, way, pixel, in, the, light, comin, ..."


In [8]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
stop = stopwords.words('english')

lyrics_by_song3.filtered = lyrics_by_song3['tokens'].apply(lambda x: [item for item in x if item not in stop])


/Users/brooklynpayne/anaconda3/envs/pandas/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [61]:

X = lyrics_by_song2.values[:, 0:4]
y = lyrics_by_song2.values[:,4]


In [42]:
from sklearn.utils import shuffle
from nltk.corpus import stopwords

genres = [
    'Country', 'Pop', 'Hip Hop', 'Rock'
]

LYRIC_LEN = 400 # each song has to be  400 characters
N = 10000 # number of records to pull from each genre
RANDOM_SEED = 200 # random seed to make results repeatable

train_df = pd.DataFrame()
test_df = pd.DataFrame()
for genre in genres: # loop over each genre
    subset = lyrics_by_song[ # create a subset 
        X = lyrics_by_song.genre==genre) & 
        (lyrics_by_song.lyric.str.len() > 400)
    ]
    train_set = subset.sample(n=N, random_state=RANDOM_SEED)
    test_set = subset.drop(train_set.index)
#     train_df = train_df.append(train_set) # append subsets to the master sets
#     test_df = test_df.append(test_set)
    
# train_df = shuffle(train_df)
# test_df = shuffle(test_df)


ValueError: 'a' must be greater than 0 unless no samples are taken

In [19]:
lyrics_by_song.sample?

In [18]:
train_df.head(10)

""


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
# define our model
text_clf = Pipeline(
    [('vect', CountVectorizer()),
     ('clf', MultinomialNB(alpha=0.1))])

In [ ]:
# train our model on training data
text_clf.fit(train_df.lyric, train_df.ranker_genre)

In [ ]:
# score our model on testing data
predicted = text_clf.predict(test_df.lyric)
np.mean(predicted == test_df.genre)